In [25]:
%autoreload
import pandas as pd
import numpy as np
import os
import re
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from dateutil import parser
from tpot import TPOTRegressor
import tensorflow as tf
import feather
from importlib import reload

from model_utilities import get_features_labels, train_random_forest, mape, complete_model, predict_mid, training_curves

ERROR:root:Line magic function `%autoreload` not found.


ImportError: cannot import name 'training_curves'

In [3]:
building_name = 'SRP'
filepath = ('python_data/f-%s_weather.csv' % building_name)

In [41]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
%autoreload
results_df, predictions_df, features_df = complete_model(filepath)

['num_time', 'ghi', 'dif', 'gti', 'temp', 'rh', 'pwat', 'ws', 'month', 'day', 'season_day', 'sun_rise_set_rise', 'sun_rise_set_set', 'biz_day_0', 'biz_day_1', 'day_of_week_Fri', 'day_of_week_Mon', 'day_of_week_Sat', 'day_of_week_Sun', 'day_of_week_Thu', 'day_of_week_Tue', 'day_of_week_Wed', 'week_day_end_day', 'week_day_end_end']
[  5.80290110e-01   2.24371309e-02   6.17855510e-03   4.24810099e-03
   1.87990987e-01   1.80291321e-02   1.44320071e-02   8.76101565e-03
   1.96554421e-03   1.00392873e-02   2.87588359e-02   9.45170170e-05
   9.80300019e-05   3.42010004e-02   4.81213621e-02   1.11294476e-03
   2.23757355e-03   2.22990661e-04   2.00308580e-04   6.72324939e-04
   1.12011017e-03   1.20468723e-03   1.72157235e-02   1.03677212e-02]
24
24


In [56]:
feather.write_dataframe(results_df, ("feather/%s_season_performance_metrics.feather" % building_name))
feather.write_dataframe(predictions_df, ("feather/%s_season_predictions.feather" % building_name))
feather.write_dataframe(features_df, ("feather/%s_feature_importances.feather" % building_name))

In [52]:
%autoreload
mid_df, metrics_mid_df = predict_mid(filepath)

In [57]:
feather.write_dataframe(metrics_mid_df, ("feather/%s_mid_performance_metrics.feather" % building_name))
feather.write_dataframe(mid_df, ("feather/%s_mid.feather" % building_name))

In [55]:
metrics_mid_df
# mid_df[mid_df['summer'] != 0]

,season,rmse,rsquared,mape
0,spring,4.393270,0.844820,8.085962
1,summer,4.871096,0.891920,7.358376
2,fall,4.897169,0.820545,8.445955
3,winter,6.930595,0.622965,11.461733


In [ ]:
results = np.array([])
for index, name in enumerate(['then', 'then']):
    print(index)
    results = results +  [2, 4]

results

In [21]:
d = {'col1': [2,3], 'col2': ['this', 'that']}
df = pd.DataFrame(data=d)

In [22]:
df

,col1,col2
0,2,this
1,3,that


In [ ]:
feather.write_dataframe(predictions_df, ('%s_predictions.feather' % 'srp'))

In [ ]:
# Parameter grid for search
rf_parameters = {
    'max_features': ['auto', 0.25, 0.5, 0.75],
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [None, 10, 20, 40],
    'min_samples_split': [2, 4, 8]
}

# Model for searching
rf_model = RandomForestRegressor()

# Random search instantiation
rf_grid_search = GridSearchCV(rf_model, rf_parameters, scoring = 'r2', cv = 2, n_jobs = -1, verbose = 2)

In [ ]:
# Read in dataframe
srp_df = pd.read_csv('python_data/f-SRP_weather.csv')

# Extract training and testing features and labels
X, y = get_features_labels(srp_df, season = 'summer')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

# Train the random search and print all the results
rf_grid_search.fit(X_train, y_train)
rf_grid_search.cv_results_

In [ ]:
rf_grid_search.best_estimator_

In [ ]:
predictions = rf_grid_search.predict(X_test)
rsquared = r2_score(y_test, predictions)
print('r squared with best estimator {:.4f}'.format(rsquared))

In [ ]:
rf_parameters = {
    'max_features': [0.5, 0.75],
    'n_estimators': [200, 250, 300],
    'max_depth': [20, 40],
    'min_samples_split': [2]
}

rf_model = RandomForestRegressor()
rf_grid_search = GridSearchCV(rf_model, rf_parameters, scoring = 'r2', cv = 2, n_jobs = -1, verbose = 2)

rf_grid_search.fit(X_train, y_train)
rf_grid_search.cv_results_

In [ ]:
predictions = rf_grid_search.predict(X_test)
rsquared = r2_score(y_test, predictions)
print('r squared with best estimator {:.4f}'.format(rsquared))

In [ ]:
rf_grid_search.best_estimator_

In [ ]:
rf_grid_search.best_params_

In [ ]:
import os
os.getcwd()

In [5]:
import random


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [23]:
df = pd.read_csv(filepath)
X, y = get_features_labels(df, season = 'summer')

results_df = pd.DataFrame(columns = ['season', 'set', 'rmse', 'rsquared', 'mape_result', 
                                     'fraction', 'train_points', 'test_points'])

index = 0
seasons = ['spring', 'summer', 'fall', 'winter']

# Iterate through seasons for 20 different lengths of training data
for season in seasons:
    print(season)
    X, y = get_features_labels(df, season = season)

    for fraction in np.arange(0.05, 1.05, 0.05):
        if fraction == 0.55:
            print('Halfway there!')

        sample = np.random.permutation(len(X))
        # Length based on i from 0.05 to 1.00 in 0.05 increments
        subset_length = int(fraction * len(X))

        # Each time the X and y are randomly mixed and then subsetted
        X_mixed, y_mixed = X[sample][:subset_length], y[sample][:subset_length]

        # Train test split on the subset with the appropriate test size
        X_train_subset, X_test, y_train_subset, y_test = train_test_split(X_mixed, y_mixed, test_size = 0.3)

        # Length of training and testing datasets
        train_length = len(X_train_subset)
        test_length = len(X_test)

        # Create and train the model
        model = RandomForestRegressor(n_estimators = 200, max_depth = 40)
        model.fit(X_train_subset, y_train_subset)

        # Predict on the test data and evaluate training performance
        train_predictions = model.predict(X_train_subset)
        train_rmse = np.sqrt(mean_squared_error(y_train_subset, train_predictions))
        train_rsquared = r2_score(y_train_subset,  train_predictions)
        train_mape = mape(y_train_subset, train_predictions)
        train_length = len(X_train_subset)

        # Add training results to dataframe
        results_df.loc[index] = [season, 'train', train_rmse, 
        train_rsquared, train_mape, fraction, train_length, test_length]

        # Increment dataframe index
        index += 1

        # Predict on the test data and evaluate test performance
        test_predictions = model.predict(X_test)
        test_rmse = np.sqrt(mean_squared_error(y_test, test_predictions))
        test_rsquared = r2_score(y_test, test_predictions)
        test_mape = mape(y_test, test_predictions)
        test_length = len(X_test)

        # Add test results to dataframe
        results_df.loc[index] = [season, 'test', test_rmse, test_rsquared,
        test_mape, fraction, train_length, test_length]

        # Increment dataframe index
        index += 1

spring
Halfway there!
summer
Halfway there!
fall
Halfway there!
winter
Halfway there!


In [37]:
train_floats = [float(points) for points in results_df['train_points']]
test_floats = [float(points) for points in results_df['test_points']]
results_df['train_points'] = train_floats
results_df['test_points'] = test_floats

In [38]:
train_floats

[610.0,
 610.0,
 1221.0,
 1221.0,
 1831.0,
 1831.0,
 2443.0,
 2443.0,
 3053.0,
 3053.0,
 3664.0,
 3664.0,
 4274.0,
 4274.0,
 4886.0,
 4886.0,
 5496.0,
 5496.0,
 6107.0,
 6107.0,
 6717.0,
 6717.0,
 7329.0,
 7329.0,
 7939.0,
 7939.0,
 8550.0,
 8550.0,
 9160.0,
 9160.0,
 9772.0,
 9772.0,
 10382.0,
 10382.0,
 10993.0,
 10993.0,
 11603.0,
 11603.0,
 12215.0,
 12215.0,
 618.0,
 618.0,
 1236.0,
 1236.0,
 1854.0,
 1854.0,
 2472.0,
 2472.0,
 3091.0,
 3091.0,
 3709.0,
 3709.0,
 4327.0,
 4327.0,
 4945.0,
 4945.0,
 5563.0,
 5563.0,
 6182.0,
 6182.0,
 6800.0,
 6800.0,
 7418.0,
 7418.0,
 8036.0,
 8036.0,
 8654.0,
 8654.0,
 9273.0,
 9273.0,
 9891.0,
 9891.0,
 10509.0,
 10509.0,
 11127.0,
 11127.0,
 11746.0,
 11746.0,
 12364.0,
 12364.0,
 611.0,
 611.0,
 1223.0,
 1223.0,
 1835.0,
 1835.0,
 2447.0,
 2447.0,
 3059.0,
 3059.0,
 3670.0,
 3670.0,
 4282.0,
 4282.0,
 4894.0,
 4894.0,
 5506.0,
 5506.0,
 6118.0,
 6118.0,
 6729.0,
 6729.0,
 7341.0,
 7341.0,
 7953.0,
 7953.0,
 8565.0,
 8565.0,
 9177.0,
 9177.0,


In [39]:
feather.write_dataframe(results_df, 'training.feather')

In [40]:
read_results = feather.read_dataframe('training.feather')

In [41]:
read_results.tail()

,season,set,rmse,rsquared,mape_result,fraction,train_points,test_points
155,winter,test,2.022800,0.953341,3.716288,0.90,10946.0,4692.0
156,winter,train,0.749451,0.993532,1.373939,0.95,11554.0,4953.0
157,winter,test,2.011937,0.954810,3.760084,0.95,11554.0,4953.0
158,winter,train,0.737207,0.993840,1.345644,1.00,12163.0,5213.0
159,winter,test,1.975315,0.954659,3.623982,1.00,12163.0,5213.0
